# Create an ingress controller with a static public IP
This example shows how to deploy the HAProxy ingress controller in an Azure Kubernetes Service (AKS) cluster. The ingress controller is configured with a static public IP address. The cert-manager project is used to automatically generate and configure Let's Encrypt certificates. Finally, applications are run in the AKS cluster, each of which is accessible over a single FQDN (Fully Qualified Domain Name) address.
## Prerequisites
* [Install cert-manager](https://github.com/amy88ma/Ingress-Configuration/blob/0deef09a5e74184e43fcac84e0b40289543e72db/Jupyter%20Notebooks/Install_cert-manager.ipynb)
* [Install helm](https://github.com/amy88ma/Ingress-Configuration/blob/0deef09a5e74184e43fcac84e0b40289543e72db/Jupyter%20Notebooks/Install_helm.ipynb)
* Have an existing AKS cluster 
* Run Azure CLI version 2.0.64

## Create an ingress controller
By default, an HAProxy ingress controller is created with a new public IP address assignment. This public IP address is only static for the life-span of the ingress controller, and is lost if the controller is deleted and re-created. A common configuration requirement is to provide the ingress controller an existing static public IP address. The static public IP address remains if the ingress controller is deleted. This approach allows you to use existing DNS records and network configurations in a consistent manner throughout the lifecycle of your applications.

If you need to create a static public IP address, first get the resource group name of the AKS cluster with the az aks show command:

In [ ]:
$ az aks show --resource-group $RESOURCE_GROUP_NAME --name $CLUSTER_NAME --query nodeResourceGroup -o tsv

The output which shows the resource group name is shown:

In [ ]:
MC_ingress-amy_aks-ingress-amy-eastus2_eastus2

Next, create a public IP address with the static allocation method using the az network public-ip create command. The following example creates a public IP address named myAKSPublicIP in the AKS cluster resource group obtained in the previous step:

In [ ]:
az network public-ip create --resource-group MC_ingress-amy_aks-ingress-amy-eastus2_eastus2 --name myAKSPublicIP --sku Standard --allocation-method static --query publicIp.ipAddress -o tsv

The output shows the static public IP address created:

In [ ]:
20.97.220.225

Now deploy the HAProxy-ingress chart with Helm. For added redundancy, two replicas of the ingress controllers are deployed with the --set controller.replicaCount parameter. To fully benefit from running replicas of the ingress controller, make sure there's more than one node in your AKS cluster.

You must pass two additional parameters to the Helm release so the ingress controller is made aware both of the static IP address of the load balancer to be allocated to the ingress controller service, and of the DNS name label being applied to the public IP address resource. For the HTTPS certificates to work correctly, a DNS name label is used to configure an FQDN for the ingress controller IP address.

1. Add the --set controller.service.loadBalancerIP parameter. Specify your own public IP address that was created in the previous step.
2. Add the --set controller.service.annotations."service\.beta\.kubernetes\.io/azure-dns-label-name" parameter. Specify a DNS name label to be applied to the public IP address that was created in the previous step.

In [ ]:
helm install haproxy-ingress haproxy-ingress/haproxy-ingress\
>   --create-namespace --namespace ingress-controller\
>   --version 0.12.6\
>   --set controller.replicaCount=2 \
>   --set controller.service.loadBalancerIP="20.97.220.225" \
>   --set controller.service.annotations."service\.beta\.kubernetes\.io/azure-dns-label-name"="mypublicip"

The output shows HAProxy controller was installed:

In [ ]:
NAME: haproxy-ingress
LAST DEPLOYED: Sun Jul 25 22:45:27 2021
NAMESPACE: ingress-controller
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
HAProxy Ingress has been installed!

When the Kubernetes load balancer service is created for the ingress controller, your static IP address is assigned, as shown in the following example output:

In [ ]:
kubectl --namespace ingress-controller get services haproxy-ingress -o wide -w

In [ ]:
NAME              TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)                      AGE   SELECTOR
haproxy-ingress   LoadBalancer   10.0.62.167   20.97.220.225   80:32724/TCP,443:32741/TCP   60s   app.kubernetes.io/instance=haproxy-ingress,app.kubernetes.io/name=haproxy-ingress

You can verify that the DNS name label has been applied by querying the FQDN on the public IP address as follows:

In [ ]:
az network public-ip list --resource-group MC_ingress-amy_aks-ingress-amy-eastus2_eastus2 --query "[?name=='myAKSblicIP'].[dnsSettings.fqdn]" -o tsv

In [ ]:
mypublicip.eastus2.cloudapp.azure.com

The ingress controller is now accessible through the IP address or the FQDN.

## Create a CA cluster issuer
Before certificates can be issued, cert-manager requires an Issuer or ClusterIssuer resource. These Kubernetes resources are identical in functionality, however Issuer works in a single namespace, and ClusterIssuer works across all namespaces. For more information, see the cert-manager issuer documentation.

Create a cluster issuer, such as cluster-issuer.yaml, using the following example manifest. Update the email address with a valid address from your organization:

In [ ]:
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-staging
spec:
  acme:
    server: https://acme-staging-v02.api.letsencrypt.org/directory
    email: user@example.com
    privateKeySecretRef:
      name: letsencrypt-staging
    solvers:
    - http01:
        ingress:
          class: haproxy
          podTemplate:
            spec:
              nodeSelector:
                "kubernetes.io/os": linux

In [ ]:
$ kubectl apply -f cluster-issuer.yaml

clusterissuer.cert-manager.io/letsencrypt-staging created

## Create an ingress route
In the following example, traffic to the address https://mypublicip.eastus2.cloudapp.azure.com/ followed by path name is routed to the specific service.  Update the hosts and host to the DNS name you created in a previous step.

Create a file named hpcc-ingress.yaml and copy in the following example.

In [ ]:
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: hpcc-ingress
  annotations:
    kubernetes.io/ingress.class: haproxy
    nginx.ingress.kubernetes.io/use-regex: "true"
    nginx.ingress.kubernetes.io/rewrite-target: /$1
    cert-manager.io/cluster-issuer: letsencrypt
spec:
  tls:
  - hosts:
    - mypublicip.eastus2.cloudapp.azure.com  
    secretName: tls-secret
  rules:
  - host: mypublicip.eastus2.cloudapp.azure.com  
    http:
      paths:
      - backend:
          serviceName: eclwatch
          servicePort: 8010
        path: /eclwatch(/|$)(.*)
      - backend:
          serviceName: eclqueries
          servicePort: 8002
        path: /eclqueries(/|$)(.*)
      - backend:
          serviceName: esdl-sandbox
          servicePort: 8899
        path: /esdl(/|$)(.*)
      - backend:
          serviceName: sql2ecl
          servicePort: 8510
        path: /wssql(/|$)(.*)
      - backend:
          serviceName: eclservices
          servicePort: 8010
        path: /(.*)

Create the ingress resource using the ```kubectl apply``` command:

In [ ]:
$ kubectl apply -f hpcc-ingress.yaml

ingress.networking.k8s.io/hpcc-ingress configured

## Verify certificate object
Next, a certificate resource must be created. The certificate resource defines the desired X.509 certificate. For more information, see cert-manager certificates.

To verify that the certificate was created successfully, use the ```kubectl describe certificate tls-secret --namespace ingress-basic``` command.

If the certificate was issued, you will see output similar to the following:

In [ ]:
Type    Reason          Age   From          Message
----    ------          ----  ----          -------
  Normal  CreateOrder     11m   cert-manager  Created new ACME order, attempting validation...
  Normal  DomainVerified  10m   cert-manager  Domain "demo-aks-ingress.eastus.cloudapp.azure.com" verified with "http-01" validation
  Normal  IssueCert       10m   cert-manager  Issuing certificate...
  Normal  CertObtained    10m   cert-manager  Obtained certificate from ACME server
  Normal  CertIssued      10m   cert-manager  Certificate issued successfully

## Test the ingress configuration
Open a web browser to the FQDN of your Kubernetes ingress controller, such as https://mypublicip.eastus2.cloudapp.azure.com.

As these examples use letsencrypt-staging, the issued TLS/SSL certificate is not trusted by the browser. Accept the warning prompt to continue to your application. The certificate information shows this Fake LE Intermediate X1 certificate is issued by Let's Encrypt. This fake certificate indicates cert-manager processed the request correctly and received a certificate from the provider.

When you change Let's Encrypt to use ```prod``` rather than ```staging```, a trusted certificate issued by Let's Encrypt is used.

Delete the previously created cluster issuer file, and change ```staging``` to ```prod```:

In [ ]:
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-staging
spec:
  acme:
    server: https://acme-staging-v02.api.letsencrypt.org/directory
    email: user@example.com
    privateKeySecretRef:
      name: letsencrypt-prod
    solvers:
    - http01:
        ingress:
          class: haproxy
          podTemplate:
            spec:
              nodeSelector:
                "kubernetes.io/os": linux

Then apply this new file:

In [ ]:
$ kubectl apply -f cluster-issuer.yaml

clusterissuer.cert-manager.io/letsencrypt-staging created

## Clean up resources
1. Delete the HAProxy controller and all created resources under the namespace:

In [ ]:
$ helm uninstall haproxy-ingress --namespace ingress-controller
release "haproxy-ingress" uninstalled

2. Delete the cluster issuer file:

In [ ]:
kubectl delete -f cluster-issuer.yaml

3. Delete the ingress route file:

In [ ]:
kubectl delete -f hpcc-ingress.yaml